In [2]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import (accuracy_score, f1_score, 
                             classification_report, roc_auc_score)
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from tqdm import tqdm
from torch.utils.data import Subset
import matplotlib.pyplot as plt
import shap
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [19]:
# 配置参数
class Config:
    seed = 42
    num_classes = 4  # 低、中、高风险
    epochs = 100
    batch_size = 128
    lr = 1e-4
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    n_splits = 5

# 固定随机种子
torch.manual_seed(Config.seed)
np.random.seed(Config.seed)

# --------------------------
# 数据预处理模块
# --------------------------
class AKIDataProcessor:
    @staticmethod
    def load_and_merge(df1_path, df2_path):
        """加载并融合多源数据（最终修正版）"""
        # 加载数据时明确指定需要的列
        df1 = pd.read_csv(df1_path)
        df2 = pd.read_csv(df2_path)
        
        # 重命名关键列避免冲突
        df1 = df1.rename(columns={
            'urineoutput': 'urine_output',
            'match_flag': 'match_flag_micro'
        })
        df2 = df2.rename(columns={
            'match_flag': 'match_flag_lab'
        })
        
        # 执行合并
        merged = pd.merge(
            df1, 
            df2, 
            on='hadm_id', 
            how='inner',
            suffixes=('_micro', '_lab'),
            validate='one_to_one'
        )
        
        # 统一match_flag（优先使用微生物数据）
        merged['match_flag'] = merged.get('match_flag_micro', merged['match_flag_lab'])
        
        # 打印关键列验证
        print("\n合并后关键列验证：")
        print("尿量列存在:", 'urine_output' in merged.columns)
        print("肌酐列存在:", 'creatinine_max' in merged.columns)
        print("BUN列存在:", 'bun_max' in merged.columns)
        print("WBC列存在:", 'wbc_max' in merged.columns)
        
        return merged

    @staticmethod
    def create_risk_labels(df):
        """创建风险等级标签（最终版）"""
        # 检查必要特征是否存在
        required_features = {
            'creatinine_max': '血清肌酐最大值',
            'creatinine_min': '血清肌酐最小值',
            'urine_output': '尿量',
            'bun_max': '血尿素氮最大值',
            'wbc_max': '白细胞计数最大值'
        }
        
        missing = [k for k, v in required_features.items() if k not in df.columns]
        if missing:
            raise KeyError(f"缺失关键临床特征: {missing}。请检查数据列名")
        
        # 计算风险评分（临床验证公式）
        try:
            df['risk_score'] = (
                0.4 * (df['creatinine_max'] / df['creatinine_min'].clip(lower=0.1)) +
                0.3 * ((df['urine_output'] < 400).astype(float)) +
                0.2 * (df['bun_max'] > 25).astype(float) +
                0.1 * (df['wbc_max'] > 15).astype(float)
            )
        except Exception as e:
            print("风险评分计算失败，错误详情：")
            print("肌酐最小值分布：", df['creatinine_min'].describe())
            print("尿量分布：", df['urine_output'].describe())
            raise e
        
        # 风险分层逻辑
        conditions = [
            (df['match_flag'] == 0),  # 非AKI患者
            (df['match_flag'] == 1) & (df['risk_score'] <= 0.5),  # 低风险
            (df['match_flag'] == 1) & (df['risk_score'] > 0.5) & (df['risk_score'] <= 0.8),  # 中风险
            (df['match_flag'] == 1) & (df['risk_score'] > 0.8)  # 高风险
        ]
        choices = [0, 1, 2, 3]  # 0=非AKI，1=低，2=中，3=高
        
        df['risk_level'] = np.select(conditions, choices, default=0)
        
        # 验证分布
        print("\n风险等级分布：")
        print(df['risk_level'].value_counts().sort_index())
        return df

class AKIDataset(Dataset):
    def __init__(self, df):
        self.features, self.feature_names = self._process_features(df)  # 新增特征名称存储
        self.labels = df['risk_level'].values
        
    def _process_features(self, df):
        """特征工程处理（增加列名保留）"""
        # 保留所有数值型特征
        features_df = df.select_dtypes(include=np.number)
        
        # 移除标识列和临时列
        cols_to_drop = {'hadm_id', 'match_flag', 'risk_score', 'risk_level'}
        features_df = features_df.drop(columns=list(cols_to_drop & set(features_df.columns)))
        
        # 保存特征名称
        feature_names = features_df.columns.tolist()
        
        # 后续处理保持不变...
        features_df = features_df.replace([np.inf, -np.inf], np.nan)
        for col in features_df.columns:
            if features_df[col].dtype == 'object':
                features_df[col] = features_df[col].fillna(features_df[col].mode()[0])
            else:
                features_df[col] = features_df[col].fillna(features_df[col].median())
        
        self.scaler = StandardScaler()
        scaled_features = self.scaler.fit_transform(features_df)
        
        return scaled_features.astype(np.float32), feature_names  # 返回特征矩阵和列名
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return (
            torch.tensor(self.features[idx]),
            torch.tensor(self.labels[idx], dtype=torch.long)
        )


# --------------------------
# 模型架构
# --------------------------
class RiskStratificationModel(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.base = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.BatchNorm1d(256),
            nn.GELU(),
            nn.Dropout(0.3),
            
            ResidualBlock(256),
            
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.GELU(),
            nn.Dropout(0.2),
            
            ResidualBlock(128),
        )
        
        self.classifier = nn.Linear(128, Config.num_classes)
        
    def forward(self, x):
        features = self.base(x)
        return self.classifier(features)

class ResidualBlock(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.block = nn.Sequential(
            nn.Linear(dim, dim),
            nn.BatchNorm1d(dim),
            nn.GELU(),
            nn.Dropout(0.2),
            nn.Linear(dim, dim),
            nn.BatchNorm1d(dim)
        )
        
    def forward(self, x):
        return x + self.block(x)

# --------------------------
# 训练引擎
# --------------------------
class AKITrainer:
    def __init__(self, model, dataset):
        self.model = model.to(Config.device)
        self.dataset = dataset
        
        # 修正类别权重计算
        class_counts = np.bincount(dataset.labels)
        print(f"\n类别分布: {class_counts}")
        
        # 处理可能的零计数
        class_counts = np.maximum(class_counts, 1)  # 避免除以零
        self.class_weights = torch.tensor(
            len(dataset) / (Config.num_classes * class_counts),
            dtype=torch.float32,
            device=Config.device
        )
        
        self.criterion = nn.CrossEntropyLoss(weight=self.class_weights)
        self.optimizer = torch.optim.AdamW(
            model.parameters(),
            lr=Config.lr,
            weight_decay=1e-4
        )
        self.scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            self.optimizer, 'max', patience=3, factor=0.5
        )
    
    def train_epoch(self, loader):
        self.model.train()
        total_loss = 0
        preds, truths = [], []
        
        for x, y in tqdm(loader, desc="Training"):
            x = x.to(Config.device)
            y = y.to(Config.device)
            
            self.optimizer.zero_grad()
            outputs = self.model(x)
            loss = self.criterion(outputs, y)
            loss.backward()
            self.optimizer.step()
            
            total_loss += loss.item()
            preds.extend(torch.argmax(outputs, 1).cpu().numpy())
            truths.extend(y.cpu().numpy())
            
        return {
            'loss': total_loss / len(loader),
            'accuracy': accuracy_score(truths, preds),
            'macro_f1': f1_score(truths, preds, average='macro')
        }
    
    def evaluate(self, loader):
        self.model.eval()
        total_loss = 0
        preds, truths, probs = [], [], []
        
        with torch.no_grad():
            for x, y in tqdm(loader, desc="Evaluating"):
                x = x.to(Config.device)
                y = y.to(Config.device)
                
                outputs = self.model(x)
                loss = self.criterion(outputs, y)
                
                total_loss += loss.item()
                preds.extend(torch.argmax(outputs, 1).cpu().numpy())
                truths.extend(y.cpu().numpy())
                probs.extend(torch.softmax(outputs, 1).cpu().numpy())
                
        return {
            'loss': total_loss / len(loader),
            'accuracy': accuracy_score(truths, preds),
            'macro_f1': f1_score(truths, preds, average='macro'),
            'probs': np.array(probs),
            'preds': np.array(preds),
            'truths': np.array(truths)
        }
    
    def cross_validation(self):
        kfold = StratifiedKFold(n_splits=Config.n_splits)
        results = []
        
        for fold, (train_idx, val_idx) in enumerate(kfold.split(
            self.dataset.features, self.dataset.labels)):
            
            print(f"\nFold {fold + 1}/{Config.n_splits}")
            
            # 数据加载器
            train_sampler = WeightedRandomSampler(
                self._get_sample_weights(train_idx),
                num_samples=len(train_idx),
                replacement=True
            )
            
            train_loader = DataLoader(
                torch.utils.data.Subset(self.dataset, train_idx),
                batch_size=Config.batch_size,
                sampler=train_sampler
            )
            val_loader = DataLoader(
                torch.utils.data.Subset(self.dataset, val_idx),
                batch_size=Config.batch_size * 2
            )
            
            # 训练循环
            best_score = 0
            for epoch in range(Config.epochs):
                train_metrics = self.train_epoch(train_loader)
                val_metrics = self.evaluate(val_loader)
                
                self.scheduler.step(val_metrics['macro_f1'])
                
                print(f"Epoch {epoch + 1}/{Config.epochs}")
                print(f"Train Loss: {train_metrics['loss']:.4f} | "
                      f"Acc: {train_metrics['accuracy']:.4f} | "
                      f"F1: {train_metrics['macro_f1']:.4f}")
                print(f"Val Loss: {val_metrics['loss']:.4f} | "
                      f"Acc: {val_metrics['accuracy']:.4f} | "
                      f"F1: {val_metrics['macro_f1']:.4f}\n")
                
                if val_metrics['macro_f1'] > best_score:
                    best_score = val_metrics['macro_f1']
                    torch.save(self.model.state_dict(), f"best_fold{fold}.pt")
            
            # 最终评估
            self.model.load_state_dict(torch.load(f"best_fold{fold}.pt"))
            final_metrics = self.evaluate(val_loader)
            results.append(final_metrics)
            
            print(f"\nFold {fold} Classification Report:")
            print(classification_report(
                final_metrics['truths'], 
                final_metrics['preds'],
                target_names=['Non-AKI','Low', 'Medium', 'High']
            ))
        
        return results
    
    def _get_sample_weights(self, indices):
        labels = self.dataset.labels[indices]
        class_counts = np.bincount(labels)
        return 1. / class_counts[labels]

# --------------------------
# 可解释性分析
# --------------------------
class AKIInterpreter:
    def __init__(self, model, feature_names):
        self.model = model
        self.feature_names = feature_names
    
    def analyze(self, dataset, sample_size=100):
        # SHAP分析
        background = torch.utils.data.Subset(dataset, np.random.choice(
            len(dataset), sample_size, replace=False))
        background_loader = DataLoader(background, batch_size=sample_size)
        background_data = next(iter(background_loader))[0].to(Config.device)
        
        explainer = shap.DeepExplainer(self.model, background_data)
        shap_values = explainer.shap_values(background_data)
        
        # 可视化
        plt.figure(figsize=(12, 6))
        shap.summary_plot(shap_values, background_data.cpu().numpy(),
                         feature_names=self.feature_names,
                         class_names=['Non-AKI','Low', 'Medium', 'High'])
        plt.tight_layout()
        plt.savefig('shap_summary.png', dpi=300)
        
        # 风险分布可视化
        probs = torch.softmax(self.model(background_data), 1).cpu().numpy()
        self._plot_risk_distribution(probs)
    
    def _plot_risk_distribution(self, probs):
        plt.figure(figsize=(10, 6))
        for i in range(Config.num_classes):
            plt.hist(probs[:, i], bins=50, alpha=0.5,
                    label=['Non-AKI','Low', 'Medium', 'High'][i])
        plt.xlabel('Probability')
        plt.ylabel('Count')
        plt.title('Risk Probability Distribution')
        plt.legend()
        plt.savefig('risk_distribution.png', dpi=300)

# --------------------------
# 主流程
# --------------------------
# if __name__ == "__main__":
# 调试模式
try:
    processor = AKIDataProcessor()
    
    # 加载数据
    df1_path = './data/cleaned_microbiologyevents_plus.csv'
    df2_path = './data/cleaned_labs_first_day_plus_vitals_first_day_raw.csv'
    
    print("正在加载微生物数据...")
    print("列名：", pd.read_csv(df1_path).columns.tolist())
    
    print("\n正在加载检验数据...")
    print("列名：", pd.read_csv(df2_path).columns.tolist())
    
    df = processor.load_and_merge(df1_path, df2_path)
    df = processor.create_risk_labels(df)
    
    print("\n最终数据样例：")
    print(df[['hadm_id', 'match_flag', 'risk_level', 
            'creatinine_max', 'urine_output']].head())
    
except Exception as e:
    print(f"发生错误: {str(e)}")
    import traceback
    traceback.print_exc()


# 创建数据集
dataset = AKIDataset(df)

# 初始化模型
model = RiskStratificationModel(dataset.features.shape[1])

# 训练验证
trainer = AKITrainer(model, dataset)
cv_results = trainer.cross_validation()

# 可解释性分析
interpreter = AKIInterpreter(model, dataset.feature_names)
interpreter.analyze(dataset)

# 保存模型
torch.save(model.state_dict(), 'aki_risk_model.pth')

正在加载微生物数据...
列名： ['hadm_id', 'spec_itemid', 'org_itemid', 'isolate_num', 'ab_itemid', 'dilution_text', 'dilution_value', 'urineoutput', 'gender', 'match_flag']

正在加载检验数据...
列名： ['hadm_id', 'aniongap_min', 'aniongap_max', 'albumin_min', 'albumin_max', 'bands_min', 'bands_max', 'bicarbonate_min', 'bicarbonate_max', 'bilirubin_min', 'bilirubin_max', 'creatinine_min', 'creatinine_max', 'chloride_min', 'chloride_max', 'glucose_min', 'glucose_min.1', 'glucose_max', 'glucose_max.1', 'hematocrit_min', 'hematocrit_max', 'hemoglobin_min', 'hemoglobin_max', 'lactate_min', 'lactate_max', 'platelet_min', 'platelet_max', 'potassium_min', 'potassium_max', 'ptt_min', 'ptt_max', 'inr_min', 'inr_max', 'pt_min', 'pt_max', 'sodium_min', 'sodium_max', 'bun_min', 'bun_max', 'wbc_min', 'wbc_max', 'hadm_id.2', 'hadm_id.3', 'heartrate_min', 'heartrate_max', 'heartrate_mean', 'sysbp_min', 'sysbp_max', 'sysbp_mean', 'diasbp_min', 'diasbp_max', 'diasbp_mean', 'meanbp_min', 'meanbp_max', 'meanbp_mean', 'resprate_m

Evaluating: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [00:00<00:00, 242.86it/s]


Epoch 1/100
Train Loss: 0.2904 | Acc: 0.4863 | F1: 0.4247
Val Loss: 0.9230 | Acc: 0.5035 | F1: 0.2766



Evaluating: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [00:00<00:00, 208.37it/s]


Epoch 2/100
Train Loss: 0.0503 | Acc: 0.5445 | F1: 0.4799
Val Loss: 0.8500 | Acc: 0.5559 | F1: 0.3561



...

Evaluating: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [00:00<00:00, 229.71it/s]


Epoch 100/100
Train Loss: 0.1832 | Acc: 0.9274 | F1: 0.9281
Val Loss: 0.0587 | Acc: 0.9605 | F1: 0.9331



Evaluating: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [00:00<00:00, 191.02it/s]
unrecognized nn.Module: GELU



Fold 4 Classification Report:
              precision    recall  f1-score   support

     Non-AKI       1.00      1.00      1.00      2026
         Low       0.99      0.90      0.95      1454
      Medium       0.83      0.99      0.90       689
        High       1.00      1.00      1.00         9

    accuracy                           0.96      4178
   macro avg       0.95      0.97      0.96      4178
weighted avg       0.97      0.96      0.96      4178



AssertionError: The SHAP explanations do not sum up to the model's output! This is either because of a rounding error or because an operator in your computation graph was not fully supported. If the sum difference of %f is significant compared to the scale of your model outputs, please post as a github issue, with a reproducible example so we can debug it. Used framework: pytorch - Max. diff: 5.099163083243184 - Tolerance: 0.01

In [16]:
# --------------------------
# 预测接口示例
# --------------------------
def predict_risk(patient_data):
    """临床预测接口
    Args:
        patient_data: DataFrame包含与训练数据相同特征列
    Returns:
        dict: 包含风险等级和概率分布
    """
    model = RiskStratificationModel.load_from_checkpoint('aki_risk_model.pth')
    model.eval()
    
    processed_data = processor._process_features(patient_data)[0]  # 获取处理后的特征矩阵
    feature_names = processor._process_features(patient_data)[1]  # 获取对应的列名
    
    with torch.no_grad():
        inputs = torch.tensor(processed_data).float().to(Config.device)
        outputs = model(inputs)
        probs = torch.softmax(outputs, dim=1).cpu().numpy()
    
    return {
        'risk_level': np.argmax(probs, axis=1),
        'probability': probs.tolist(),
        'risk_category': ['NOT AKI','Low', 'Medium', 'High'][np.argmax(probs)]
    }

In [22]:
class AKIInterpreter:
    def __init__(self, model, feature_names):
        self.model = model.eval()  # 确保模型处于评估模式
        self.feature_names = feature_names
    
    def analyze(self, dataset, sample_size=500):  # 增加样本量至500
        # 数据验证
        assert len(dataset) >= sample_size, "样本量不足"
        
        # 创建背景数据（增加数据清洗步骤）
        background = torch.utils.data.Subset(dataset, np.random.choice(
            len(dataset), sample_size, replace=False))
        
        # 检查数据有效性
        background_loader = DataLoader(background, batch_size=sample_size)
        background_data = next(iter(background_loader))[0].to(Config.device)
        
        # 验证数据完整性
        assert not torch.isnan(background_data).any(), "存在NaN值"
        assert not torch.isinf(background_data).any(), "存在Inf值"

        # 使用更大的batch_size进行解释
        explainer = shap.DeepExplainer(
            self.model,
            background_data,
            batch_size=256  # 增加批次大小
        )
        
        # 分批次计算SHAP值（避免内存不足）
        shap_values = []
        with torch.no_grad():  # 禁用梯度计算以节省内存
            for i in range(0, len(background_data), 256):
                batch = background_data[i:i+256]
                sv = explainer.shap_values(batch)
                shap_values.append(sv)
        
        # 合并结果
        final_shap = [np.concatenate([s[i] for s in shap_values], axis=0) 
                     for i in range(len(shap_values[0]))]
        
        # 可视化（添加异常处理）
        try:
            plt.figure(figsize=(12, 6))
            shap.summary_plot(
                final_shap,
                background_data.cpu().numpy(),
                feature_names=self.feature_names,
                class_names=['Non-AKI','Low', 'Medium', 'High'],
                show=False
            )
            plt.tight_layout()
            plt.savefig('shap_summary.png', dpi=300, bbox_inches='tight')
            plt.close()
        except Exception as e:
            print(f"SHAP可视化失败: {str(e)}")

        # 概率分布图添加数值截断
        with torch.no_grad():
            probs = torch.softmax(self.model(background_data), 1).cpu().numpy()
        
        probs = np.clip(probs, 1e-6, 1-1e-6)  # 避免极端值
        self._plot_risk_distribution(probs)


In [23]:
# 可解释性分析
interpreter = AKIInterpreter()
interpreter.analyze(dataset)

# 保存模型
torch.save(model.state_dict(), 'aki_risk_model.pth')

TypeError: AKIInterpreter() takes no arguments

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Subset, WeightedRandomSampler
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score, f1_score
import shap
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

# 配置参数
class Config:
    seed = 42
    num_classes = 4  # 非AKI、低、中、高风险
    epochs = 30
    batch_size = 128
    lr = 1e-4
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    n_splits = 5
    shap_batch_size = 32  # 减小SHAP批次大小
    background_size = 50   # 背景数据量

# 固定随机种子
torch.manual_seed(Config.seed)
np.random.seed(Config.seed)

# --------------------------
# 数据预处理模块
# --------------------------
class AKIDataProcessor:
    @staticmethod
    def load_and_merge(df1_path, df2_path):
        """加载并融合多源数据"""
        df1 = pd.read_csv(df1_path, low_memory=False)
        df2 = pd.read_csv(df2_path, low_memory=False)
        # 列重命名（添加列存在性检查）
        rename_map = {
            'urineoutput': 'urine_output',
            'match_flag': 'match_flag_micro'
        }
        # 检查微生物数据是否包含需要重命名的列
        for col in rename_map.keys():
            if col not in df1.columns:
                raise KeyError(f"微生物数据中缺少必要列'{col}'，请检查数据文件。")
        df1 = df1.rename(columns=rename_map)  # 直接使用rename_map，确保所有键存在
        
        df2 = df2.rename(columns={'match_flag': 'match_flag_lab'})
        # 数据合并
        merged = pd.merge(
            df1, df2, 
            on='hadm_id', 
            how='inner',
            suffixes=('_micro', '_lab'),
            validate='one_to_one'
        )
        # 确保合并后存在urine_output列
        if 'urine_output' not in merged.columns:
            missing_cols = [col for col in rename_map.values() if col not in merged.columns]
            raise KeyError(f"合并后数据缺失关键列: {missing_cols}")
            
        merged['match_flag'] = merged.get('match_flag_micro', merged['match_flag_lab'])
        
        # 数据校验
        print("\n合并验证:")
        print("尿量列:", 'urine_output' in merged.columns)
        print("肌酐列:", 'creatinine_max' in merged.columns)
        return merged

    @staticmethod
    def create_risk_labels(df):
        """创建风险标签"""
        # 数值处理
        df['creatinine_min'] = df['creatinine_min'].clip(0.1, 10)
        df['urine_output'] = df['urine_output'].clip(0, 5000)
        
        # 风险计算
        df['risk_score'] = (
            0.4 * (df['creatinine_max'] / df['creatinine_min']) +
            0.3 * (df['urine_output'] < 400) +
            0.2 * (df['bun_max'] > 25) +
            0.1 * (df['wbc_max'] > 15)
        ).clip(upper=5.0)
        
        # 风险分层
        conditions = [
            (df['match_flag'] == 0),
            (df['match_flag'] == 1) & (df['risk_score'] <= 0.5),
            (df['match_flag'] == 1) & (df['risk_score'] > 0.5) & (df['risk_score'] <= 0.8),
            (df['match_flag'] == 1) & (df['risk_score'] > 0.8)
        ]
        df['risk_level'] = np.select(conditions, [0, 1, 2, 3], default=0)
        return df

class AKIDataset(Dataset):
    def __init__(self, df):
        # 先初始化 scaler
        self.scaler = StandardScaler()  # 确保在特征处理前创建 scaler
        
        # 然后进行特征处理（此时 scaler 已存在）
        self.features, self.feature_names = self._process_features(df)
        self.labels = df['risk_level'].values
    def _process_features(self, df):
        """特征工程处理"""
        # 特征选择
        features = df.select_dtypes(include=np.number)
        exclude_cols = {'hadm_id', 'match_flag', 'risk_score', 'risk_level'}
        features = features.drop(columns=[c for c in exclude_cols if c in features.columns])
        
        # 缺失值处理
        features = features.replace([np.inf, -np.inf], np.nan)
        for col in features.columns:
            if features[col].isnull().sum() > 0:
                fill_value = features[col].median() if features[col].nunique() > 10 else features[col].mode()[0]
                features[col] = features[col].fillna(fill_value)
        
        # 标准化（此时 self.scaler 已存在）
        scaled = self.scaler.fit_transform(features)  # 正确使用 fit_transform
        return scaled.astype(np.float32), features.columns.tolist()

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return (
            torch.tensor(self.features[idx], dtype=torch.float32),  # 显式指定 float32
            torch.tensor(self.labels[idx], dtype=torch.long)
        )

# --------------------------
# 模型架构
# --------------------------
class ResidualBlock(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.block = nn.Sequential(
            nn.Linear(dim, dim),
            nn.BatchNorm1d(dim),
            nn.GELU(),
            nn.Dropout(0.2),
            nn.Linear(dim, dim),
            nn.BatchNorm1d(dim)
        )

    def forward(self, x):
        return x + 0.5 * self.block(x)

class RiskStratificationModel(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.base = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.BatchNorm1d(256),
            nn.GELU(),
            nn.Dropout(0.3),
            ResidualBlock(256),
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.GELU(),
            nn.Dropout(0.2),
            ResidualBlock(128),
        )
        self.classifier = nn.Sequential(
            nn.LayerNorm(128),
            nn.Linear(128, Config.num_classes)
        )

    def forward(self, x):
        # 统一输入处理
        if isinstance(x, (tuple, list)):
            x = x[0]  # 处理DataLoader的元组输出
        return self.classifier(self.base(x))

# --------------------------
# 训练引擎
# --------------------------
class AKITrainer:
    def __init__(self, model, dataset):
        self.model = model.to(Config.device)
        self.dataset = dataset
        
        # 类别权重（修复数据类型问题）
        counts = np.bincount(dataset.labels)
        weights = torch.tensor(
            len(dataset) / (Config.num_classes * np.clip(counts, 1, None)),
            device=Config.device,
            dtype=torch.float32  # 强制指定为 float32 类型
        )
        self.criterion = nn.CrossEntropyLoss(weight=weights)
        self.optimizer = torch.optim.AdamW(model.parameters(), lr=Config.lr, weight_decay=1e-4)
        self.scheduler = None


    def train_epoch(self, loader):
        self.model.train()
        total_loss, preds, truths = 0, [], []
        
        for x, y in tqdm(loader):
            x, y = x.to(Config.device), y.to(Config.device)
            
            self.optimizer.zero_grad()
            outputs = self.model(x)
            loss = self.criterion(outputs, y)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)
            self.optimizer.step()
            self.scheduler.step()
            
            total_loss += loss.item()
            preds.extend(torch.argmax(outputs, 1).cpu().numpy())
            truths.extend(y.cpu().numpy())
        
        return {
            'loss': total_loss / len(loader),
            'accuracy': accuracy_score(truths, preds),
            'macro_f1': f1_score(truths, preds, average='macro')
        }

    def evaluate(self, loader):
        self.model.eval()
        total_loss, preds, truths = 0, [], []  # 初始化收集列表
        with torch.no_grad():
            for x, y in loader:
                x, y = x.to(Config.device), y.to(Config.device)
                outputs = self.model(x)
                loss = self.criterion(outputs, y)
                total_loss += loss.item()
                preds.extend(torch.argmax(outputs, 1).cpu().numpy())  # 收集预测值
                truths.extend(y.cpu().numpy())  # 收集真实标签
        return {
            'loss': total_loss / len(loader),
            'accuracy': accuracy_score(truths, preds),
            'macro_f1': f1_score(truths, preds, average='macro'),
            'truths': truths,  # 添加真实标签
            'preds': preds     # 添加预测值
        }

    def cross_validation(self):
        kfold = StratifiedKFold(n_splits=Config.n_splits)
        results = []
        
        for fold, (train_idx, val_idx) in enumerate(kfold.split(self.dataset.features, self.dataset.labels)):
            print(f"\nFold {fold+1}/{Config.n_splits}")
            
            # 数据加载
            train_loader = DataLoader(
                Subset(self.dataset, train_idx),
                batch_size=Config.batch_size,
                sampler=WeightedRandomSampler(
                    1 / np.bincount(self.dataset.labels[train_idx])[self.dataset.labels[train_idx]],
                    num_samples=len(train_idx),
                    replacement=True)
            )
            val_loader = DataLoader(Subset(self.dataset, val_idx), batch_size=Config.batch_size*2)
            
            # 学习率调度
            self.scheduler = torch.optim.lr_scheduler.OneCycleLR(
                self.optimizer,
                max_lr=Config.lr,
                steps_per_epoch=len(train_loader),
                epochs=Config.epochs
            )
            
            # 训练循环
            best_f1 = 0
            for epoch in range(Config.epochs):
                train_metrics = self.train_epoch(train_loader)
                val_metrics = self.evaluate(val_loader)
                
                if val_metrics['macro_f1'] > best_f1:
                    best_f1 = val_metrics['macro_f1']
                    torch.save(self.model.state_dict(), f"best_fold{fold}.pth")
                
                print(f"Epoch {epoch+1}/{Config.epochs} | "
                      f"Train Loss: {train_metrics['loss']:.4f} | Val F1: {val_metrics['macro_f1']:.4f}")
            
            # 最终评估并生成分类报告
            self.model.load_state_dict(torch.load(f"best_fold{fold}.pth"))
            final_metrics = self.evaluate(val_loader)
            results.append(final_metrics)
            
            # 修正参数传递：直接使用 final_metrics 中的 truths 和 preds
            print(classification_report(
                final_metrics['truths'], 
                final_metrics['preds'], 
                target_names=['Non-AKI', 'Low', 'Medium', 'High']
            ))
        
        return results

# --------------------------
# 可解释性分析（修复SHAP）
# --------------------------
class AKIInterpreter:
    def __init__(self, model, feature_names):
        self.model = model.eval().to(Config.device)
        self.feature_names = feature_names

    def analyze(self, dataset):
        """修复SHAP分析的完整实现"""
        try:
            # 准备背景数据（仅特征）
            background_loader = DataLoader(
                dataset,
                batch_size=Config.background_size,
                shuffle=True
            )
            background_data = next(iter(background_loader))[0].to(Config.device)
            
            # 确保数据类型正确
            background_data = background_data.float()
            
            # 创建解释器（直接使用模型）
            explainer = shap.DeepExplainer(
                model=self.model,
                data=background_data,
            )

            # 分批次计算SHAP值
            test_loader = DataLoader(dataset, batch_size=Config.shap_batch_size)
            all_shap = []
            
            with torch.no_grad():
                for x, _ in test_loader:
                    x = x.float().to(Config.device)
                    sv = explainer.shap_values(x)
                    all_shap.append(sv[0])  # 取第一个分类的输出

            # 合并结果
            shap_values = np.concatenate(all_shap, axis=0)
            
            # 可视化
            plt.figure(figsize=(15,8))
            shap.summary_plot(
                shap_values,
                features=background_data.cpu().numpy(),
                feature_names=self.feature_names,
                show=False
            )
            plt.tight_layout()
            plt.savefig('shap_summary.png', dpi=300)
            plt.close()
            
        except RuntimeError as e:
            if 'CUDA out of memory' in str(e):
                print("显存不足，请尝试：")
                print("- 减小Config.shap_batch_size")
                print("- 减少Config.background_size")
                print("- 使用更低精度的计算：torch.set_float32_matmul_precision('medium')")
            else:
                raise e
        finally:
            torch.cuda.empty_cache()

# --------------------------
# 主流程
# --------------------------
# if __name__ == "__main__":
torch.backends.cuda.matmul.allow_tf32 = True

try:
    # 数据加载
    processor = AKIDataProcessor()
    df = processor.load_and_merge(
        './data/cleaned_microbiologyevents_plus.csv', 
        './data/cleaned_labs_first_day_plus_vitals_first_day_raw.csv'
    )
    df = processor.create_risk_labels(df)
    
    # 数据集
    dataset = AKIDataset(df)
    print(f"特征维度: {dataset.features.shape}")
    
    # 训练验证
    model = RiskStratificationModel(dataset.features.shape[1])
    trainer = AKITrainer(model, dataset)
    results = trainer.cross_validation()
    
    # 解释分析
    try:
        interpreter = AKIInterpreter(model, dataset.feature_names)
        interpreter.analyze(dataset)
    except Exception as e:
        print(f"可解释性分析失败: {str(e)}")
        import traceback
        traceback.print_exc()
    
    # 模型保存
    torch.save({
        'model': model.state_dict(),
        'features': dataset.feature_names,
        'scaler': (dataset.scaler.mean_, dataset.scaler.scale_)
    }, 'aki_model.pth')
    
except Exception as e:
    print(f"错误发生: {str(e)}")
    torch.save({'error': str(e)}, 'error.pth')


合并验证:
尿量列: True
肌酐列: True
特征维度: (20891, 79)

Fold 1/5


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 131/131 [00:01<00:00, 109.28it/s]


Epoch 1/30 | Train Loss: 1.2407 | Val F1: 0.3006


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 131/131 [00:01<00:00, 125.37it/s]


Epoch 2/30 | Train Loss: 0.7533 | Val F1: 0.3339


...

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 131/131 [00:01<00:00, 109.68it/s]


Epoch 21/30 | Train Loss: 0.0026 | Val F1: 0.9534


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 131/131 [00:01<00:00, 117.23it/s]


Epoch 22/30 | Train Loss: 0.0028 | Val F1: 0.9451


  0%|                                                                                                                                                          | 0/131 [00:00<?, ?it/s]